Основная задача: проанализировать выборку страховых событий (ДТП с двумя участниками) на возможное мошенничество.  Выделить тех клиентов, относительно которых существует подозрение на мошеннические действия; обосновать подозрение.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

Загружаем и смотрим на данные

In [2]:
data = pd.read_csv('data.csv', encoding = 'cp1251', sep = ';', usecols = [0,1,2])
data.head()

,№ страхового события,Участник 1,Участник 2
0,1,Беляев Владислав Аркадьевич,Ефимов Малик Константинович
1,2,Давыдова Ануш Оскаровна,Никифорова Божена Львовна
2,3,Белов Сергей Михайлович,Третьяков Никита Харитонович
3,4,Власов Артём Михайлович,Аксенова Вероника Кузьминична
4,5,Яковлев Алан Макарович,Сысоев Елисей Тимофеевич


Получаем список уникальных имен

In [3]:
def unuque_names(data):
    names = data['Участник 1']
    names = names.append(data['Участник 2'])
    names = list(set(names))
    return names
names = unuque_names(data)
names

['Павлова Мария Геннадиевна',
 'Лукина Аделия Георгиевна',
 'Игнатова Эльвира Геннадиевна',
 'Селезнева Амелия Михайловна',
 'Ершова Эвелина Тарасовна',
 'Панова Екатерина Никифоровна',
 'Рыбаков Богдан Давидович',
 'Кабанова Таисия Сергеевна',
 'Федотов Ростислав Богуславович',
 'Шубина Куралай Эдуардовна',
 'Тимофеева Эльвира Юрьевна',
 'Крылова Таисия Ждановна',
 'Ковалева Злата Борисовна',
 'Гаврилов Даниил Ефимович',
 'Самсонова Амелия Аскольдовна',
 'Кононов Давлат Артемович',
 'Шестаков Вольга Дмитриевич',
 'Сысоев Елисей Тимофеевич',
 'Фомин Ратибор Феликсович',
 'Федотова Людмила Степановна',
 'Кудряшова Стася Борисовна',
 'Воронцов Георгий Олегович',
 'Степанова Милослава Ильинична',
 'Гришина Рузана Рудольфовна',
 'Мясникова Злата Степановна',
 'Титова Елена Анатольевна',
 'Игнатьева Марина Захаровна',
 'Князева Юлия Макаровна',
 'Харитонов Радислав Богданович',
 'Поляков Мирослав Анатольевич',
 'Фролова Юлиана Владимировна',
 'Артемьев Доброслав Платонович',
 'Михеева Софья

In [4]:
groups = []
# для каждого человека получаем данные о всех авариях с участием данного человека
for name in names: 
    df_names = data[(data['Участник 1'] == name) | (data['Участник 2'] == name)]
    
    # если аварий больше одной, то получаем список других участников с данных человеком
    if len(df_names) > 1:
        names_list = unuque_names(df_names)
        df_group = pd.DataFrame()
        # получаем список аварий других участников
        for n in names_list:
            df_group = df_group.append(data[(data['Участник 1'] == n) | (data['Участник 2'] == n) ])
        df_group = df_group.drop_duplicates()
        group = unuque_names(df_group)
        # если участники повторяются, то хохраняем их имена
        if len(df_group) > (len(group)-1):
            groups.append(sorted(group))

In [5]:
# удаляем повторяющиеся
import itertools
groups = sorted(groups)
groups = list(groups for groups,_ in itertools.groupby(groups))

In [6]:
# отображаем группы и записываем имена
names = []
for group in groups:
    df_group = pd.DataFrame()
    for n in group:
        df_group = df_group.append(data[(data['Участник 1'] == n) | (data['Участник 2'] == n) ])
    df_group = df_group.drop_duplicates()
    names.append(unuque_names(df_group))
    display(df_group) 
with open('names.txt', 'w') as f_out:
    for group in names:
        for name in group:
            f_out.write(name + '\n')

,№ страхового события,Участник 1,Участник 2
89,90,Воробьев Иван Александрович,Комин Сергей Николаевич
142,143,Павлова Мария Геннадиевна,Воробьев Иван Александрович
62,63,Комин Сергей Николаевич,Павлова Мария Геннадиевна


,№ страхового события,Участник 1,Участник 2
136,137,Мухамадеев Александр Валерьевич,Коробов Вадим Александрович
184,185,Коробов Вадим Александрович,Рогачев Антон Владимирович
49,50,Мухамадеев Александр Валерьевич,Сенчукова Екатерина Семеновна
115,116,Рогачев Антон Владимирович,Сенчукова Екатерина Семеновна


В нескольких авариях учавствовали одни и теже люди в разных комбинациях, что довольно подозрительно.
Среди этих людей возможен факт мошенничества.